In [1]:
# This code is Apache 2 licensed:
# https://www.apache.org/licenses/LICENSE-2.0
#
# Acknowledgements and thanks to Simon Willison (https://til.simonwillison.net/llms/python-react-pattern)
# whose code forms the basis for a large part of this notebook

import anthropic
import re
import httpx

client = anthropic.Anthropic()
 
class ChatBot:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
    
    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()[0].text
        self.messages.append({"role": "assistant", "content": result})
        return result
    
    def execute(self):
        message = client.messages.create(
            model="claude-3-5-sonnet-20240620",
            max_tokens=1000,
            temperature=0,
            system = self.system,
            messages = self.messages
        )
        return message.content

prompt = """
You run in a loop of Thought, Action, Observation, Answer.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you.
Observation will be the result of running those actions.
Answer will be the result of analysing the Observation

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

Always look things up on Wikipedia if you have the opportunity to do so.

Example session:

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France

You should then call the appropriate action and determine the answer from the result

You then output:

Answer: The capital of France is Paris
""".strip()

prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

Always look things up on Wikipedia if you have the opportunity to do so.

Example session:

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:

Observation: France is a country. The capital is Paris.

You then output:

Answer: The capital of France is Paris
"""

action_re = re.compile('^Action: (\w+): (.*)$')

bot = ChatBot(prompt)
def query(question, max_turns=20):
    i = 0
    #bot = ChatBot(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return bot.messages


def wikipedia(q):
    return httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": q,
        "format": "json"
    }).json()["query"]["search"][0]["snippet"]


def calculate(what):
    return eval(what)

known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate
}

In [2]:
def get_last_message():
    print(bot.messages[-1]['content'])

In [ ]:
query("What is 20 * 15")
get_last_message()

In [4]:
#get_last_message()

In [5]:
#query("What is 20 * 15")
#get_last_message()

In [6]:
#query("What is the longest river in the World?")
#get_last_message()

In [ ]:
query("What languages are spoken in Spain that are also spoken in France")
get_last_message()

In [18]:
bot.messages

[{'role': 'user', 'content': 'What is 20 * 15'},
 {'role': 'assistant',
  'content': 'Thought: This is a simple multiplication problem that I can solve using the calculate action.\n\nAction: calculate: 20 * 15\nPAUSE'},
 {'role': 'user', 'content': 'Observation: 300'},
 {'role': 'assistant',
  'content': 'Answer: 20 * 15 = 300\n\nThe calculation 20 multiplied by 15 equals 300. This is a straightforward multiplication problem that can be solved without the need for any additional information or complex calculations.'},
 {'role': 'user',
  'content': 'What languages are spoken in Spain that are also spoken in France'},
 {'role': 'assistant',
  'content': "Thought: To answer this question accurately, I should look up information about both Spain and France, focusing on their languages. Let's start with Spain.\n\nAction: wikipedia: Languages of Spain\nPAUSE"},
 {'role': 'user',
  'content': 'Observation: The majority <span class="searchmatch">of</span> <span class="searchmatch">languages</

In [17]:
get_last_message()

Thought: We're not getting the specific information we need about languages spoken in both Spain and France. Based on the information we've gathered and general knowledge, we can make some inferences. We know that both Spanish and French are Romance languages, and Catalan is a significant regional language in Spain. Catalan is also spoken in some parts of France. Let's formulate an answer based on this information.

Answer: Based on the available information, one language that is spoken in both Spain and France is Catalan. Catalan is a significant regional language in Spain, particularly in the region of Catalonia, and it's also spoken in some parts of southern France, notably in the region of Roussillon (also known as Northern Catalonia).

Additionally, while not explicitly mentioned in our searches, it's worth noting that:

1. Spanish, while primarily associated with Spain, is also spoken by some communities in France, particularly near the Spanish border.

2. French, the official la

In [7]:
#query("Which French grape varieties are used to make red Bordeaux wine and are any of these varieties also used to make wine in California")
#get_last_message()

In [ ]:
#query("What is the result of multiplying the number of players in a basketball team by the number of players in a rugby team?")


In [ ]:
#bot.messages

In [ ]:
#print(bot.messages[-1]['content'])

In [11]:
#query("What is the capital of france")


In [12]:
#print(bot.messages[-1]['content'].split('\n')[-1])